In [1]:
import pandas as pd
import numpy as np

# Domain
x = 200
z = 100
y = 50+30 # 50m for the Moon and 30m for the Air

# Depth of Heterogeneity
d = 10

# Frequency and Wavelength
f = 2*50e6 # Maximum Relevant Frequency for the Ricker Wavelet
c = 3e8

# Permittivity and Density
density = 2 # g/cm^3
eps_bg = 1.919**density # Background Permittivity for Moon
vel_bg = c / eps_bg**0.5

lambda_0 = c / (f * eps_bg**0.5)

# # Fresnel Zone - Radius
# R = 0.5 * (lambda_0 * d)**0.5
# y = 4 * R

# Grid Descretization
dx = np.round(lambda_0 / 12, 3)
dy = np.round(lambda_0 / 12, 3)
dz = np.round(lambda_0 / 12, 3)

nx = int(x / dx)
ny = int(y / dy)
nz = int(z / dz)
print(f"nx: {nx}, ny: {ny}, nz: {nz}")

# x_grid, y_grid, z_grid = np.meshgrid(x_vals, y_vals, z_vals, indexing='xy')

# Time Window
time_window = 2 * y / vel_bg

# Print
print(f"Domain: {x} x {y} x {z}")
print(f"Depth of Heterogeneity: {d}")
print(f"Base Permittivity of Moon: {eps_bg}")
print(f"Frequency: {f/2}")
print(f"Wavelength: {lambda_0}")
# print(f"Fresnel Zone Radius: {R}")
print(f"Grid: {dx} x {dy} x {dz}")
print(f"Time Window: {time_window}")


nx: 1538, ny: 615, nz: 769
Domain: 200 x 80 x 100
Depth of Heterogeneity: 10
Base Permittivity of Moon: 3.682561
Frequency: 50000000.0
Wavelength: 1.5633142261594581
Grid: 0.13 x 0.13 x 0.13
Time Window: 1.0234666666666666e-06


In [2]:
# Editing the Lava Tube Points to Comply with the Domain

data_all = pd.read_csv('../LavaTubeData/LiDAR_InclineCave_TUBE_TLS_points.txt', sep=' ', header=None)
# Convert to numpy arrays X, Y, Z
data_points = data_all.to_numpy()
X = data_points[:, 0]
Y = data_points[:, 2]
Z = data_points[:, 1]

# Find the minimum and maximum values of X, Y, Z, and find their index

X_min_index = X.argmin() 
X_min = X[X_min_index]
X_max_index = X.argmax()
X_max = X[X_max_index]
Y_min_index = Y.argmin()
Y_min = Y[Y_min_index]
Y_max_index = Y.argmax()
Y_max = Y[Y_max_index]
Z_min_index = Z.argmin()
Z_min = Z[Z_min_index]
Z_max_index = Z.argmax()
Z_max = Z[Z_max_index]

# Shift all points in X, Y, Z to the center of the domain
X = X - X_min
Y = Y - Y_min
Z = Z - Z_min

X_min_index = X.argmin() 
X_min = X[X_min_index]
X_max_index = X.argmax()
X_max = X[X_max_index]
Y_min_index = Y.argmin()
Y_min = Y[Y_min_index]
Y_max_index = Y.argmax()
Y_max = Y[Y_max_index]
Z_min_index = Z.argmin()
Z_min = Z[Z_min_index]
Z_max_index = Z.argmax()
Z_max = Z[Z_max_index]

X = X - (X_min + X_max)/2 + x/2
Y = Y - Y_max + y - d - 30
Z = Z - (Z_min + Z_max)/2 + z/2

X_min_index = X.argmin() 
X_min = X[X_min_index]
X_max_index = X.argmax()
X_max = X[X_max_index]
Y_min_index = Y.argmin()
Y_min = Y[Y_min_index]
Y_max_index = Y.argmax()
Y_max = Y[Y_max_index]
Z_min_index = Z.argmin()
Z_min = Z[Z_min_index]
Z_max_index = Z.argmax()
Z_max = Z[Z_max_index]

print(f"X_min: {X_min}")
print(f"X_max: {X_max}")
print(f"Y_min: {Y_min}")
print(f"Y_max: {Y_max}")
print(f"Z_min: {Z_min}")
print(f"Z_max: {Z_max}")

data_all_shifted = pd.DataFrame(np.column_stack((X, Y, Z)))
data_all_shifted = data_all_shifted.to_numpy()

# Save the new points to vtk
from pyevtk.hl import pointsToVTK
pointsToVTK("LiDAR_InclineCave_TUBE_adjusted", X, Y, Z)

X_min: 35.02539050002815
X_max: 164.97460949997185
Y_min: 14.520020000000159
Y_max: 40.0
Z_min: 18.73046849993989
Z_max: 81.26953150006011


'f:\\Projects\\LunarLeaper\\LLGPR\\src\\LiDAR_InclineCave_TUBE_adjusted.vtu'

In [3]:
# def uniform_resample_point_cloud(point_cloud, grid_spacing, X_min, X_max, Y_min, Y_max, Z_min, Z_max):
#     # Compute the bounding box of the point cloud
#     min_bounds = np.array([X_min, Y_min, Z_min])
#     max_bounds = np.array([X_max, Y_max, Z_max])

#     # Create a grid of points within the bounding box
#     x = np.arange(min_bounds[0], max_bounds[0], grid_spacing)
#     y = np.arange(min_bounds[1], max_bounds[1], grid_spacing)
#     z = np.arange(min_bounds[2], max_bounds[2], grid_spacing)
#     grid_points = np.array(np.meshgrid(x, y, z)).T.reshape(-1, 3)

#     # Find the closest point in the original point cloud to each grid point
#     resampled_points = []
#     for gp in grid_points:
#         # Calculate the distance from the grid point to all points in the original point cloud
#         distances = np.linalg.norm(point_cloud - gp, axis=1)
#         # Select the closest point
#         closest_point = point_cloud[np.argmin(distances)]
#         resampled_points.append(closest_point)

#     return np.array(resampled_points)

# # Example usage
# data_all_resampled = uniform_resample_point_cloud(data_all_shifted, dx, X_min, X_max, Y_min, Y_max, Z_min, Z_max)

# # Save the new points to vtk
# pointsToVTK("LiDAR_InclineCave_TUBE_resampled", data_all_resampled[:, 0], data_all_resampled[:, 2], data_all_resampled[:, 1])

In [4]:
from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator

# Example point cloud data (replace with your actual data)
permittivity_values = 7.6* np.ones_like(X)

In [5]:
#Grid
x_val = np.linspace(0, x, 10)
y_val = np.linspace(0, y, 10)
z_val = np.linspace(0, z, 8)

# # Create Delaunay triangulation
# points = np.column_stack((X, Y, Z))
# tri = Delaunay(points)

# # Create interpolator for permittivity values
# interpolator = LinearNDInterpolator(tri, permittivity_values)
# Could use batch processing to make it faster
# for ix, x in enumerate(x_val):
#     for iy, y in enumerate(y_val):
#         for iz, z in enumerate(z_val):
#             # Interpolate permittivity at current grid point
#             if tri.find_simplex([x, y, z]) >= 0:
#                 permittivity = interpolator(x, y, z)                
#             print(1,end=' ')


In [6]:
meshgrid_x, meshgrid_y, meshgrid_z = np.meshgrid(x_val, y_val, z_val, indexing='ij')
points = np.vstack((meshgrid_x.flatten(), meshgrid_y.flatten(), meshgrid_z.flatten())).T

# Create Delaunay triangulation
tri = Delaunay(np.column_stack((X, Y, Z)))

In [7]:
# Check which points are within the convex hull
simplex_indices = tri.find_simplex(points)
mask = simplex_indices >= 0

# Initialize permittivity array with default value (or NaN)
permittivity_array = 3.6 * np.ones(meshgrid_x.shape)

In [8]:
# Create interpolator for permittivity values
interpolator = LinearNDInterpolator(tri, permittivity_values)

# Interpolate permittivity values only where within convex hull
permittivity_array[mask] = interpolator(points[mask])

# Permittivity model for subsurface of moon without topography


IndexError: boolean index did not match indexed array along axis 0; size of axis is 10 but size of corresponding boolean axis is 800